In [2]:
from hyperopt import fmin, hp, tpe, Trials
import numpy as np

import os
import sys

BASEDIR = os.path.dirname(os.getcwd())
sys.path.append(BASEDIR)

from copy import deepcopy
from functools import partial
from dgl.dataloading import GraphDataLoader
from src.model.attentivefp import attentivefp
from src.utils.mol.attfp_graph import MoleculeDataset, collate_fn
from src.config.attentivefp import attentivefpArgs
from src.pipeline.ensemble import training_ensemble_models
from src.utils.basic.logger import Writer, NpEncoder
from src.utils.model.metrics import accuracy, roc_auc_score, prc_auc, F1, MCC, expected_calibration_error, OverconfidentFalseRate, OverconfidentFalseNegatives, Brier
import torch
import pandas as pd
import json

# Initialize

In [3]:
torch.set_num_threads(4)

target_name = 'CardioTox'
gpu_num     = 0

# Bayesian optimization searching space

SPACE = {
    'hidden_size'       : hp.quniform('hidden_size', low=300, high=600, q=100),
    'radius'            : hp.quniform('radius', low=2, high=6, q=1),
    'T'                 : hp.quniform('T', low=1, high=5, q=1),
    'p_dropout'         : hp.quniform('dropout', low=0.0, high=0.5, q=0.05),
    'init_lr'           : hp.loguniform('init_lr', low=np.log(1e-4), high=np.log(1e-2)),
    'ffn_num_layers'    : hp.quniform('ffn_num_layers', low=2, high=4, q=1)
}

INT_KEYS = ['hidden_size', 'radius', 'T', 'ffn_num_layers']
SAVEDIR = os.path.join(BASEDIR, "results", target_name, 'AttFp')
DATADIR = os.path.join(BASEDIR, "data", target_name)

# Training function

In [5]:
n = 0
logger = Writer(os.path.join(SAVEDIR, "history.log"))
def func(hyperparams):
    logger(" ")
    logger(" ")

    global n
    n = n+1
    logger(f"ROUND {n}")

    BASESAVEDIR = os.path.join(SAVEDIR, f"ROUND_{n}")
    logger_writer = Writer(os.path.join(BASESAVEDIR, "config.txt"))
    for k in hyperparams:
        logger_writer(f"{k}\t{hyperparams[k]}")

    for key in INT_KEYS:
        hyperparams[key] = int(hyperparams[key])

    config = attentivefpArgs().parse_args([], known_only=True)
    hyper_args = deepcopy(config)

    for key, value in hyperparams.items():
        setattr(hyper_args, key, value)

    setattr(hyper_args, "dataset_type", "classification")
    setattr(hyper_args, 'metric', "roc-auc")
    setattr(hyper_args, "extra_metrics", ["MCC", "prc-auc", "accuracy", "F1"])
    setattr(hyper_args, "ffn_hidden_size", hyper_args.hidden_size)
    setattr(hyper_args, "early_stopping_num", 30)
    setattr(hyper_args, "gpu", gpu_num)
    setattr(hyper_args, "log_frequency", 100)
    setattr(hyper_args, "batch_size", 128)
    setattr(hyper_args, "at_least_epoch", 0)
    print(hyper_args)

    for i in [1, 2, 3, 4, 5]:

        train_dataset = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_train.csv"))
        valid_dataset = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_valid.csv"))
        test_dataset  = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_test.csv"))

        train_dataloader = GraphDataLoader(dataset=train_dataset, collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=True)
        train_dataloader.smiles = [[s] for s in train_dataset.smiles_list]
        valid_dataloader = GraphDataLoader(dataset=valid_dataset, collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=False)
        valid_dataloader.smiles = [[s] for s in valid_dataset.smiles_list]
        test_dataloader  = GraphDataLoader(dataset=test_dataset,  collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=False)
        test_dataloader.smiles  = [[s] for s in test_dataset.smiles_list]

        training_ensemble_models(os.path.join(BASESAVEDIR, f"fold_{i}"),
                                 attentivefp,
                                 hyper_args,
                                 train_dataloader,
                                 valid_dataloader=valid_dataloader,
                                 test_dataloader=test_dataloader,
                                 ensemble_num=1)
        
        test_prediction = []
        
        valid_ROC = []
        valid_PRC = []
        valid_ACC = []
        valid_MCC = []
        valid_F1  = []
        valid_ECE = []
        valid_OFR = []
        valid_OFN = []
        valid_Brier = []
        
        test_ROC  = []
        test_PRC  = []
        test_ACC  = []
        test_MCC  = []
        test_F1   = []
        test_ECE  = []
        test_OFR = []
        test_OFN = []
        test_Brier = []

    for i in [1, 2, 3, 4, 5]:
    
        temp_dir = os.path.join(BASESAVEDIR, f"fold_{i}", "model_0")
        
        temp_valid_prediction = pd.read_csv(os.path.join(temp_dir, "valid_prediction.csv"))["property_pred"].to_numpy()
        temp_valid_label = pd.read_csv(os.path.join(temp_dir, "valid_prediction.csv"))["property_label"].to_numpy()
        
        temp_test_prediction = pd.read_csv(os.path.join(temp_dir, "test_prediction.csv"))["property_pred"].to_numpy()
        test_label = pd.read_csv(os.path.join(temp_dir, "test_prediction.csv"))["property_label"].to_numpy()
        test_prediction.append(temp_test_prediction) # for ensemble calculation
        
        valid_ROC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["roc-auc"].iloc[0])
        valid_PRC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["prc-auc"].iloc[0])
        valid_ACC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["accuracy"].iloc[0])
        valid_MCC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["MCC"].iloc[0])
        valid_F1.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["F1"].iloc[0])
        valid_ECE.append(expected_calibration_error(temp_valid_label, temp_valid_prediction, bins=10))
        valid_OFR.append(OverconfidentFalseRate(temp_valid_prediction, temp_valid_label))
        valid_OFN.append(OverconfidentFalseNegatives(temp_valid_prediction, temp_valid_label))
        valid_Brier.append(Brier(temp_valid_label,temp_valid_prediction))
        
        test_ROC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["roc-auc"].iloc[0])
        test_PRC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["prc-auc"].iloc[0])
        test_ACC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["accuracy"].iloc[0])
        test_MCC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["MCC"].iloc[0])
        test_F1.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["F1"].iloc[0])
        test_ECE.append(expected_calibration_error(test_label, temp_test_prediction, bins=10))
        test_OFR.append(OverconfidentFalseRate(temp_test_prediction, test_label))
        test_OFN.append(OverconfidentFalseNegatives(temp_test_prediction, test_label))
        test_Brier.append(Brier(test_label,temp_test_prediction))
        

    logger(f'ROUND {n} Valid ROC-AUC {np.mean(valid_ROC)} +/- {np.std(valid_ROC)}')
    logger(f'ROUND {n} Valid PRC-AUC {np.mean(valid_PRC)} +/- {np.std(valid_PRC)}')
    logger(f'ROUND {n} Valid ACC     {np.mean(valid_ACC)} +/- {np.std(valid_ACC)}')
    logger(f'ROUND {n} Valid MCC     {np.mean(valid_MCC)} +/- {np.std(valid_MCC)}')
    logger(f'ROUND {n} Valid F1      {np.mean(valid_F1)} +/- {np.std(valid_F1)}')
    logger(f'ROUND {n} Valid ECE     {np.mean(valid_ECE)} +/- {np.std(valid_ECE)}')
    logger(f'ROUND {n} Valid OFR     {np.mean(valid_OFR)} +/- {np.std(valid_OFR)}')
    logger(f'ROUND {n} Valid OFN     {np.mean(valid_OFN)} +/- {np.std(valid_OFN)}')
    logger(f'ROUND {n} Valid Brier   {np.mean(valid_Brier)} +/- {np.std(valid_Brier)}')
    logger(' ')
    logger(f'ROUND {n} Test ROC-AUC {np.mean(test_ROC)} +/- {np.std(test_ROC)}')
    logger(f'ROUND {n} Test PRC-AUC {np.mean(test_PRC)} +/- {np.std(test_PRC)}')
    logger(f'ROUND {n} Test ACC     {np.mean(test_ACC)} +/- {np.std(test_ACC)}')
    logger(f'ROUND {n} Test MCC     {np.mean(test_MCC)} +/- {np.std(test_MCC)}')
    logger(f'ROUND {n} Test F1      {np.mean(test_F1)} +/- {np.std(test_F1)}')
    logger(f'ROUND {n} Test ECE     {np.mean(test_ECE)} +/- {np.std(test_ECE)}')
    logger(f'ROUND {n} Test OFR     {np.mean(test_OFR)} +/- {np.std(test_OFR)}')
    logger(f'ROUND {n} Test OFN     {np.mean(test_OFN)} +/- {np.std(test_OFN)}')
    logger(f'ROUND {n} Test Brier   {np.mean(test_Brier)} +/- {np.std(test_Brier)}')
    logger(' ')
    logger(f'ROUND {n} Ensemble Test ROC-AUC {roc_auc_score(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test PRC-AUC {prc_auc(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test ACC {accuracy(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test MCC {MCC(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test F1 {F1(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test ECE {expected_calibration_error(test_label, np.mean(test_prediction, axis=0), bins=10)}')
    logger(f'ROUND {n} Ensemble Test OFR {OverconfidentFalseRate(np.mean(test_prediction, axis=0), test_label)}')
    logger(f'ROUND {n} Ensemble Test OFN {OverconfidentFalseNegatives(np.mean(test_prediction, axis=0), test_label)}')
    logger(f'ROUND {n} Ensemble Test Brier {Brier(test_label, np.mean(test_prediction, axis=0))}')
    logger(' ')

    return -np.mean(valid_ROC)



algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(func, SPACE, algo=algo, max_evals=3)  # best is a dictionary
json_str = json.dumps(best, cls=NpEncoder)  # using json to turn a dictionary to a str 
logger(json_str)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]07-14 09:05:12	 
07-14 09:05:12	 
07-14 09:05:12	ROUND 1
07-14 09:05:12	T	5.0
07-14 09:05:12	ffn_num_layers	4.0
07-14 09:05:12	hidden_size	400.0
07-14 09:05:12	init_lr	0.0006061632142200683
07-14 09:05:12	p_dropout	0.05
07-14 09:05:12	radius	5.0
{'T': 5,                                             
 'activation': 'ReLU',
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'at_least_epoch': 0,
 'atom_descriptors': None,
 'atom_descriptors_size': None,
 'atom_messages': False,
 'batch_size': 128,
 'bias': False,
 'cuda': True,
 'dataset_type': 'classification',
 'depth': 3,
 'device': device(type='cuda', index=0),
 'dropout': 0.0,
 'early_stopping_num': 30,
 'empty_cache': False,
 'explicit_h': False,
 'extra_metrics': ['MCC', 'prc-auc', 'accuracy', 'F1'],
 'features_only': False,
 'features_size': None,
 'ffn_hidden_size': 400,
 'ffn_num_layers': 4,
 'fingerprint_dim': 200,
 'gpu': 0,
 'grad_clip': None,
 'hidden_size': 400,
 'init_lr'

07-14 09:05:12	 
07-14 09:05:12	 
07-14 09:05:12	ROUND 1
07-14 09:05:12	T	5.0
07-14 09:05:12	ffn_num_layers	4.0
07-14 09:05:12	hidden_size	400.0
07-14 09:05:12	init_lr	0.0006061632142200683
07-14 09:05:12	p_dropout	0.05
07-14 09:05:12	radius	5.0
0it [00:00, ?it/s]
30it [00:00, 296.85it/s]
72it [00:00, 364.56it/s]
111it [00:00, 367.56it/s]
155it [00:00, 395.39it/s]
198it [00:00, 404.84it/s]
243it [00:00, 418.20it/s]
285it [00:00, 417.40it/s]
327it [00:00, 376.62it/s]
369it [00:00, 388.10it/s]
410it [00:01, 392.03it/s]
450it [00:01, 345.93it/s]
494it [00:01, 369.53it/s]
538it [00:01, 387.28it/s]
579it [00:01, 392.39it/s]
619it [00:01, 386.24it/s]
659it [00:01, 354.65it/s]
696it [00:01, 344.62it/s]
740it [00:01, 368.98it/s]
782it [00:02, 379.59it/s]
821it [00:02, 350.31it/s]
862it [00:02, 365.24it/s]
908it [00:02, 391.12it/s]
949it [00:02, 396.05it/s]
990it [00:02, 355.71it/s]
1027it [00:02, 359.53it/s]
1069it [00:02, 374.29it/s]
1108it [00:02, 334.78it/s]
1146it [00:03, 345.68it/s]
1186i

07-14 09:05:34	Training 0th model.


07-14 09:05:34	Training 0th model.


07-14 09:05:44	
07-14 09:05:44	Start Evaluating on Evaluation Set, EPOCH 2 BATCH 100


...

07-14 09:12:38	
07-14 09:12:38	Start Evaluating on Evaluation Set, EPOCH 83 BATCH 4200
100%|##########| 13/13 [00:00<00:00, 22.46it/s]


07-14 09:12:38	roc-auc: 0.914
07-14 09:12:38	MCC: 0.716
07-14 09:12:38	prc-auc: 0.848
07-14 09:12:38	accuracy: 0.890
07-14 09:12:38	F1: 0.790
07-14 09:12:38	Model not improve for 30 Epochs.
07-14 09:12:38	Best roc-auc Now: 0.924
07-14 09:12:38	Finish Training.
07-14 09:12:38	Best roc-auc: 0.924


07-14 09:12:38	roc-auc: 0.914
07-14 09:12:38	MCC: 0.716
07-14 09:12:38	prc-auc: 0.848
07-14 09:12:38	accuracy: 0.890
07-14 09:12:38	F1: 0.790
07-14 09:12:38	Model not improve for 30 Epochs.
07-14 09:12:38	Best roc-auc Now: 0.924
07-14 09:12:38	Finish Training.
07-14 09:12:38	Best roc-auc: 0.924
job exception: 'list' object has no attribute 'to'



  0%|          | 0/3 [07:26<?, ?trial/s, best loss=?]


AttributeError: 'list' object has no attribute 'to'